In [29]:
import driptorch as dt
import json

In [30]:
# Burn Unit Parameters
burn_unit_geojson : 'CWLStringInput' = '{"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-114.44869995117188,47.088504171925706],[-114.44470882415771,47.08745225315146],[-114.44342136383057,47.09066638416644],[-114.44496631622313,47.09236102969754],[-114.44633960723877,47.0924194647886],[-114.45281982421875,47.089205439567344],[-114.45153236389159,47.08815353464254],[-114.44869995117188,47.088504171925706]]]}}]}'
burn_unit_firing_direction : 'CWLFloatInput' = 217
burn_unit_control_line_buffer : 'CWLFloatInput' = 10
burn_unit_downwind_buffer : 'CWLFloatInput' = 20

# Igniter Parameters
# Dash length = 0 & Gap length > 0 : Point igniter. 
# Gap length = 0 & Dash length = 0: Continuous igniter
igniter1_velocity : 'CWLFloatInput' = 1
igniter1_line_dash_length : 'CWLFloatInput' = 10
igniter1_line_gap_length : 'CWLFloatInput' = 50

igniter2_velocity : 'CWLFloatInput' = 1
igniter2_line_dash_length : 'CWLFloatInput' = 10
igniter2_line_gap_length : 'CWLFloatInput' = 50

# Firing Parameters
firing_offset : 'CWLFloatInput' = 10 # Offset distance in meters from the unit boundary

# Output Parameters
output_time_offset : 'CWLIntInput' = 25 # Time offset to add to the ignition times. Defaults to 0
output_resolution : 'CWLIntInput' = 1 # Horizontal resolution of QUIC-fire domain (in meters)
output_epsg : 'CWLStringInput' = None # EPSG code for the destination projection
output_file : 'CWLFilePathOutput' = "./qf_ignition.dat"

In [31]:
# BURN UNIT

if not burn_unit_geojson:
    raise ValueError('No geojson provided for burn unit')
burn_unit_geojson = json.loads(burn_unit_geojson)

# Create a burn unit from a GeoJSON feature collection with a wind direction
burn_unit = dt.BurnUnit.from_json(burn_unit_geojson, firing_direction=burn_unit_firing_direction)

# Add Buffer Areas
if burn_unit_control_line_buffer:
    burn_unit_firing_area = burn_unit.buffer_control_line(burn_unit_control_line_buffer)
if burn_unit_downwind_buffer:
    burn_unit_firing_area = burn_unit_firing_area.buffer_downwind(burn_unit_downwind_buffer)

fuel_removal_area = burn_unit.difference(burn_unit_firing_area)

In [32]:
# PERSONNEL (IGNITERS)
ignition_crew = dt.IgnitionCrew(same_velocity=(igniter1_velocity == igniter2_velocity))
igniter1 =  dt.Igniter(igniter1_velocity, dash_length=igniter1_line_dash_length, gap_length=igniter1_line_gap_length)
igniter2 =  dt.Igniter(igniter2_velocity, dash_length=igniter2_line_dash_length, gap_length=igniter2_line_gap_length)
ignition_crew.add_igniter(igniter1)
ignition_crew.add_igniter(igniter2)

In [33]:
# FIRING TECHNIQUES
firing = dt.firing.Ring(burn_unit_firing_area, ignition_crew)
firing_pattern = firing.generate_pattern(
    offset=firing_offset
)

In [34]:
# OUTPUTS
if not firing_pattern:
    raise ValueError('No firing pattern generated')

# Write the pattern to a QUIC-Fire ignition file
qf_ignition_file = firing_pattern.to_quicfire(
    burn_unit, 
    filename=output_file, 
    time_offset=output_time_offset,
    resolution=output_resolution,    
    dst_epsg=output_epsg
)